<a href="https://colab.research.google.com/github/ShoSato-047/DSCI330_Lab-3.1---Text-Classification-with-NLP/blob/main/DSCI330_lab3_1_text_classification_w_NLP_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = red>**BoW vs. TF-IDF:**
- Use BoW when you need a simple word frequency representation.
- Use TF-IDF when you want to weigh words based on importance across documents.

In [1]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 1.9 MB/s eta 0:00:00


In [2]:
%pip install composable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: toolz
    Found existing installation: toolz 0.12.1
    Uninstalling toolz-0.12.1:
      Successfully uninstalled toolz-0.12.1


In [3]:
import os
import composable_records as rec
import composable_tuples as tup

from composable import pipeable
from composable.strict import map, filter
from composable_utility import apply, identity, get
from composable_object import obj

from composable_glob import glob
from composable_utility import get, with_open, identity, apply

# You may need to install utility.py to use "with_open"

In [4]:
# this is the root directory
!pwd

/content


## Loading the emotions dataset

The following data set include a large number of sentences combined with a classification of the emotion.

In [5]:
from datasets import load_dataset

emotions = load_dataset("emotion")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [7]:
(train_ds := emotions >> get('train')) #getting training set

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [8]:
train_ds.column_names

['text', 'label']

In [9]:
train_ds.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [10]:
train_ds[:5]

{'text': ['i didnt feel humiliated',
  'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'im grabbing a minute to post i feel greedy wrong',
  'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'i am feeling grouchy'],
 'label': [0, 0, 3, 2, 3]}

### Preparing the dataset

In [11]:
(documents :=
 train_ds
 >> get('text')
) >> tup.head(5)

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
 'i am feeling grouchy']

In [12]:
(labels :=
 train_ds
 >> get('label')
) >> tup.head(5)

[0, 0, 3, 2, 3]

### Bag of Words

## **Bag of Words:**
In Bag of Words (BoW), the higher the value, the more frequently a word appears in a document.

The score is not limited to a range of 0 to 1. The values represent raw word counts, meaning they can be any non-negative integer starting from 0 and increasing based on how often a word appears in a document.

- Minimum: 0 (if the word is absent in a document)
- Maximum: No fixed upper limit (depends on how frequently a word appears)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create the sparse feature set
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(documents)

# Create a training and test (validation) set
X_train, X_test, y_train, y_test = train_test_split(X, labels,
                                                    test_size = 0.2,
                                                    random_state=42)

# Train the model
classifier = MultinomialNB()

classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = classifier.predict(X_test)

(accuracy := accuracy_score(y_test, y_pred))

0.7375

### TF-IDF

## **TF-IDF Formula:**

**TF-IDF = TF × IDF**

Where:

- Term Frequency (TF): Measures how often a term appears in a document.
- Inverse Document Frequency (IDF): Measures how rare a term is across documents.

Note:
- Minimum Score: 0 - If a word does not appear in a document, its score is 0.
- If *TfidfVectorizer* uses **L2 normalization** (default in scikit-learn), TF-IDF values are scaled to range between 0 and 1 for each document.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create the sparse feature set
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(documents)

# Create a training and test (validation) set
X_train, X_test, y_train, y_test = train_test_split(X, labels,
                                                    test_size = 0.2,
                                                    random_state=42)

# Train the model
classifier = MultinomialNB()

classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = classifier.predict(X_test)

(accuracy := accuracy_score(y_test, y_pred))

0.6165625

## <font color="red"> Exercise 3.6.1 </font>

The file `train.csv` contains bits of text from three classic authors in the horror genre ([source](https://www.kaggle.com/c/spooky-author-identification/)).

**Preprocessing Tasks.**
1. Read the raw lines in using `with_open`.
2. Split the data into columns and extract the text into one list and the labels into other.
3. Clean up the text by making it lower case and removing any punctuation.
4. Map the text labels to numbers.

**ML tasks.** Test the performance of the naive Bayes classifer on both the Bag of Words and TF-IDF features.

In [15]:
# Your code here

### **Step 1: Read the raw lines in using with_open**

In [16]:
# This is my current location (root)
!pwd

/content


In [17]:
(paths :=
 "/content/train.csv"
 >> glob()
)

['/content/train.csv']

In [18]:
with open(paths[0], encoding ="utf-8") as f:
    lines = f.readlines()
lines[:3]

['"id","text","author"\n',
 '"id26305","This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.","EAP"\n',
 '"id17569","It never once occurred to me that the fumbling might be a mere mistake.","HPL"\n']

### **Step 2: Split the data into columns and extract the text into one list and the labels into other**

In [19]:
# Hint: Replace ',' with '-' in each line
lines = [line.replace(', ', '- ') for line in lines]
lines[:3]

['"id","text","author"\n',
 '"id26305","This process- however- afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit- and return to the point whence I set out- without being aware of the fact; so perfectly uniform seemed the wall.","EAP"\n',
 '"id17569","It never once occurred to me that the fumbling might be a mere mistake.","HPL"\n']

In [20]:
split_lines = [line.split(',') for line in lines]
split_lines[:3]

[['"id"', '"text"', '"author"\n'],
 ['"id26305"',
  '"This process- however- afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit- and return to the point whence I set out- without being aware of the fact; so perfectly uniform seemed the wall."',
  '"EAP"\n'],
 ['"id17569"',
  '"It never once occurred to me that the fumbling might be a mere mistake."',
  '"HPL"\n']]

In [21]:
# Create a record
records = []
for row in split_lines[1:]:  # Skip header
    record = {
        "id": row[0].strip('"'),
        "text": row[1].strip('"'),
        "author": row[2].strip('"')
    }
    records.append(record)

records[:3]

[{'id': 'id26305',
  'text': 'This process- however- afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit- and return to the point whence I set out- without being aware of the fact; so perfectly uniform seemed the wall.',
  'author': 'EAP"\n'},
 {'id': 'id17569',
  'text': 'It never once occurred to me that the fumbling might be a mere mistake.',
  'author': 'HPL"\n'},
 {'id': 'id11008',
  'text': 'In his left hand was a gold snuff box- from which- as he capered down the hill- cutting all manner of fantastic steps- he took snuff incessantly with an air of the greatest possible self satisfaction.',
  'author': 'EAP"\n'}]

In [22]:
# Create a single list of text
(text_raw :=
 [record['text'] for record in records]
)

text_raw[:3]

['This process- however- afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit- and return to the point whence I set out- without being aware of the fact; so perfectly uniform seemed the wall.',
 'It never once occurred to me that the fumbling might be a mere mistake.',
 'In his left hand was a gold snuff box- from which- as he capered down the hill- cutting all manner of fantastic steps- he took snuff incessantly with an air of the greatest possible self satisfaction.']

### **Step 3: Clean up the text by making it lower case and removing any punctuation.**

In [23]:
# Making text lowercase
text_lower = [text.lower() for text in text_raw]
text_lower[:3]

['this process- however- afforded me no means of ascertaining the dimensions of my dungeon; as i might make its circuit- and return to the point whence i set out- without being aware of the fact; so perfectly uniform seemed the wall.',
 'it never once occurred to me that the fumbling might be a mere mistake.',
 'in his left hand was a gold snuff box- from which- as he capered down the hill- cutting all manner of fantastic steps- he took snuff incessantly with an air of the greatest possible self satisfaction.']

In [24]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [30]:
# Tokenizing a list of sentence into words
from nltk.tokenize import word_tokenize

text_words = [word_tokenize(text) for text in text_lower]
text_words[:1]

[['this',
  'process-',
  'however-',
  'afforded',
  'me',
  'no',
  'means',
  'of',
  'ascertaining',
  'the',
  'dimensions',
  'of',
  'my',
  'dungeon',
  ';',
  'as',
  'i',
  'might',
  'make',
  'its',
  'circuit-',
  'and',
  'return',
  'to',
  'the',
  'point',
  'whence',
  'i',
  'set',
  'out-',
  'without',
  'being',
  'aware',
  'of',
  'the',
  'fact',
  ';',
  'so',
  'perfectly',
  'uniform',
  'seemed',
  'the',
  'wall',
  '.']]

In [26]:
# Removing any punctuation
punctuation = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~'

from string import punctuation

(punc_map := str.maketrans('', '', punctuation))

{33: None,
 34: None,
 35: None,
 36: None,
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,
 42: None,
 43: None,
 44: None,
 45: None,
 46: None,
 47: None,
 58: None,
 59: None,
 60: None,
 61: None,
 62: None,
 63: None,
 64: None,
 91: None,
 92: None,
 93: None,
 94: None,
 95: None,
 96: None,
 123: None,
 124: None,
 125: None,
 126: None}

In [29]:
# Removing punctuation from each word in the tokenized lists
text_no_punc = [[word.translate(punc_map) for word in words] for words in text_words]
text_no_punc[:1]

[['this',
  'process',
  'however',
  'afforded',
  'me',
  'no',
  'means',
  'of',
  'ascertaining',
  'the',
  'dimensions',
  'of',
  'my',
  'dungeon',
  '',
  'as',
  'i',
  'might',
  'make',
  'its',
  'circuit',
  'and',
  'return',
  'to',
  'the',
  'point',
  'whence',
  'i',
  'set',
  'out',
  'without',
  'being',
  'aware',
  'of',
  'the',
  'fact',
  '',
  'so',
  'perfectly',
  'uniform',
  'seemed',
  'the',
  'wall',
  '']]

In [32]:
(clean_words :=
 [text for text in text_raw]
 >> pipeable(lambda s: s.lower())
 >> pipeable(lambda s: s.translate(punc_map))
 >> pipeable(word_tokenize)
)[:10]

AttributeError: 'list' object has no attribute 'lower'

### **Step 4: Map the text labels to numbers.**

In [ ]:
# What is the meaning of label . . .?